In [26]:
%display latex
from alive_progress import alive_bar
from pseries_basis import *
from ore_algebra import *
n = PSBasis.n(PSBasis)

In [2]:
B = BinomialBasis(); B

Binomial basis (x) choose n

In [3]:
OE.<E> = OreAlgebra(QQ[x], ('E', lambda p : p(x=x+1), lambda p : 0))
x = OE.base().gens()[0]

In [4]:
def poly_decomp(polynomial):
    r'''
        Method that splits a polynomial into a lists of monomials and a list of coefficients indexed in 
        such a way that summing thorugh both lists gives the original polynomial.
        
        This method unifies the behavior or Univariate and Multivariate polynomials.
    '''
    from sage.rings.polynomial.polynomial_element import is_MPolynomialRing, is_PolynomialRing
    if is_PolynomialRing(polynomial.parent()):
        g = polynomial.parent().gens()[0]
        d = polynomial.degree()
        monomials = [g**i for i in range(d+1)]
        coefficients = polynomial.coefficients(False)
        # we clean the zeros to have a sparse representation
        monomials = [monomials[i] for i in range(d+1) if coefficients[i] != 0]
        coefficients = [c for c in coefficients if c != 0]
    elif is_MPolynomialRing(polynomial.parent()):
        monomials = polynomial.monomials()
        coefficients = polynomial.coefficients()
    else:
        raise TypeError("The input must be a polynomial")
    return monomials, coefficients

def apply_operator_to_seq(operator, seq):
    r'''
        Method to apply an operator to a sequence.
        
        This method will be similar to the usual __call__ method from ``ore_algebra``, but with sequences that
        are given as functions.
        
        INPUT:
        
        * ``operator``: and operator with 1 generator (the shift) over the polynomial ring `\mathbb{R}[x]`.
        * ``seq``: a sequence in functional format. This means that we can call it with integer values and we 
          obtain the values of the sequence at each point.
          
        OUTPUT:
        
        A sequence in function format.
    '''
    if len(operator.parent().gens()) > 1:
        raise TypeError()
    E = operator.parent().gens()[0]
    v = operator.parent().base().gens()[0]
    coefficients = operator.coefficients(sparse=False)
    return lambda i : sum(coefficients[j](**{str(v):i})*seq(i+j) for j in range(len(coefficients)))

def get_converted_init(seq, bas, size):
    r'''
        Method that computes new initial values after a conversion.
        
        Let `((P_{n,k})_n)_k` be a basis of the sequence space and ``seq`` the
        functional representation of a sequence `(a_n)_n`. If we express this
        sequence in the form:
        
        .. MATH::
            
            a_n = \sum_{k=0}^n P_{n,k} c_k,
            
        then the sequence `(c_k)_k` can be explicitly computed by solving a linear system.
        
        This method solves such system for a fixed amount of elements (gibven by ``size``)
    '''
    inhom = vector([seq(i) for i in range(size)])
    M = Matrix([[bas[i](j) for i in range(size)] for j in range(size)])
    return list(M.solve_right(inhom))

def required_init(operator):
    return max(-min([0]+[el[0] for el in operator.polynomial().lc().roots() if el[0] in ZZ]), operator.order())
    
def solution(operator, init):
    d = operator.order()
    required = required_init(operator)
    if len(init) < required:
        raise ValueError(f"More data ({required}) is needed")
    @lru_cache
    def __aux_sol(n):
        if n < 0:
            return 0
        elif n < operator.order():
            return init[n]
        else:
            coeffs = operator.polynomial().coefficients(False)
            lc = coeffs.pop()
            return -sum(__aux_sol(n-d+i)*coeffs[i](n-d) for i in range(operator.order()))/lc(n-d)
    return __aux_sol

def eval_ore_operator(operator, ring=None,**values):
    r'''
        Method to evaluate ore operators
        
        This method evaluate operators from ``ore_algebra`` as they are polynomials. This allows to change the name 
        of the generators to try a iterative approach.
    '''
    gens = [str(el) for el in operator.parent().gens()]
    outer_vals = {el : values.get(el, 0) for el in gens}
    inner_vals = {el : values[el] for el in values if (not el in outer_vals)}
    monomials,coefficients = poly_decomp(operator.polynomial())
    coefficients = [el(**inner_vals) for el in coefficients]
    monomials = [prod(
        outer_vals[str(g)]**(m.degree(g)) for g in operator.polynomial().parent().gens()
    ) for m in monomials]
    result = sum(coefficients[i]*monomials[i] for i in range(len(monomials)))
    if ring != None:
        return ring(result)
    return result

def ass_recurrence(operator, basis, ring):
    return eval_ore_operator(basis.remove_Sni(basis.recurrence(operator)), ring, Sn=E, n=x,Sni=1)

class latex_str:
    def __init__(self, data):
        self.__data = data
        self.__latex = None
        
    def __repr__(self):
        return self.__data
    def __str__(self):
        return self.__data
    def _latex_(self):
        if self.__latex is None:
            self.__latex = self.__data.replace("*", "").replace("+0", "")
            
        return self.__latex

def print_recurrence(operator):
    r'''
        Method to print nicely the recurrence equation induced by an operator.
    '''
    if len(operator.parent().gens()) > 1:
        raise TypeError()
    E = operator.parent().gens()[0]
    v = operator.parent().base().gens()[0]
    coefficients = operator.coefficients(sparse=False)
    
    return latex_str(" + ".join(
        "(" + 
        str(coefficients[j]).replace(str(v),'n') + 
        ")*"
        "a_{n+" + str(j) + "}" for j in range(len(coefficients))) + " = 0")

In [5]:
# Catalan sequence
cat = lambda n : catalan_number(n)
cat_op = (x+2)*E - (4*x + 2)

In [6]:
cat_bin = ass_recurrence(cat_op, B, OE); show(cat_bin)
cat_bin_bin = ass_recurrence(cat_bin, B, OE); cat_bin_bin

(x + 3)*E^2 + (-2*x - 2)*E - 3*x - 3

(x + 4)*E^3 + (x + 5)*E^2 + (-4*x - 6)*E - 4*x - 4

In [7]:
in_cat = solution(cat_bin, get_converted_init(cat, B, 3))
in_in_cat = solution(cat_bin_bin, get_converted_init(in_cat, B, 4))

In [8]:
[in_in_cat(i) for i in range(10)]

[1, -1, 2, -3, 6, -10, 20, -35, 70, -126]

In [9]:
[cat(i) == sum(sum(in_in_cat(k2)*binomial(k,k2) for k2 in range(k+1))*binomial(i,k) for k in range(i+1)) for i in range(10)]

[True, True, True, True, True, True, True, True, True, True]

$$a_0 = \binom{0}{0} c_0 = c_0$$
$$a_1 = \sum_{k=0}^1 \binom{1}{k} c_k = \binom{1}{0} c_0 + \binom{1}{1} c_1 = c_0 + c_1$$

In [10]:
[cat(i) for i in range(10)]

[1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862]

In [11]:
B.remove_Sni((B.recurrence(cat_op)))

(n + 3)*Sn^2 + (-2*n - 2)*Sn - 3*n - 3

In [12]:
cat_bin

(x + 3)*E^2 + (-2*x - 2)*E - 3*x - 3

In [13]:
v = vector([1,2,3])
M = Matrix([[1,2,3],[4,5,6],[7,8,9]])

In [14]:
M*M.solve_right(v) == v

True

$$f(x) \in \mathbb{K}[[x]],\qquad f(x) = \sum_n c_n P_n(x)$$

$$(f_n)_n \in \mathbb{K}^\mathbb{N}$$

In [15]:
B

Binomial basis (x) choose n

In [16]:
[x*B[i] - (i+1)*B[i+1] - i*B[i] for i in range(2,10)]

[0, 0, 0, 0, 0, 0, 0, 0]

In [17]:
B[5]*5

1/24*x^5 - 5/12*x^4 + 35/24*x^3 - 25/12*x^2 + x

In [18]:
AUX = PolynomialRing(QQ, 'p')

In [19]:
poly = AUX([1,2,3])
monomials = poly.monomials(); monomials.reverse()
coefficients = poly.coefficients(sparse=False)

In [20]:
sum(monomials[i]*coefficients[i] for i in range(len(monomials))) == poly

True

In [21]:
AUX = PolynomialRing(QQ, 'a,b')

In [22]:
poly = AUX.random_element(3)

### Trying to find examples in OEIS of double sum

In [23]:
def explore_example(operator, init, ring, first_bases, second_bases, bound_guess=100):
    r'''
        Method to explore how to write a solution for a recurrence equation in a double sum
        
        This method takes the solution `(a_n)_n` or the recurrence equation defined by ``operator``
        and ``init`` (see method :func:`solution`) and computes a new sequence `(y_n)_n` such that
        
        .. MATH::
        
            y_n = \sum_{k} \sum_{l} a_l B_{2,l}(k) B_{1,k}(n)
            
        Then it would be easy to check whether this sequence is useful for us or not.
        
        The arguments ``first_bases`` and ``second_bases`` provide lists for `B_{1,k}(n)` and `B_{2,l}(k)`. One output for each 
        pair of basis will be computed. If these entries are just one element instead of a list or tuple, then we consider the 
        input as a one element list.
        
        INPUT:
        
        * ``operator``: a difference operator valid for the use of :func:`solution`
        * ``init``: initial values for the sequence (see :func:`solution`)
        * ``first_bases``: list of :class:`PSBasis` compatible with the recurrence operator.
        * ``second_bases``: list of :class:`PSBasis` compatible with the recurrence operator.
    '''
    ## Checking the input
    if not isinstance(first_bases, (list, tuple)):
        first_bases = [first_bases]
    if not isinstance(second_bases, (list, tuple)):
        second_bases = [second_bases]
        
    if any(not isinstance(B, PSBasis) for B in first_bases+second_bases):
        raise TypeError("All basis must be of class PSBasis")
    if any(not B.has_compatibility('E') for B in first_bases+second_bases):
        raise TypeError("All basis must be compatible with 'E'")
    
    sequence = solution(operator, init)
    
    result = {}
    for B1 in first_bases:
        for B2 in second_bases:
            b1_sequence = lambda n : sum(B1[k](n)*sequence(k) for k in range(n+1))
            b2_b1_sequence = lambda n : sum(B2[k](n)*b1_sequence(k) for k in range(n+1))
            try:
                L = guess([b2_b1_sequence(i) for i in range(bound_guess)], ring)
                result[(B1,B2)] = (b2_b1_sequence, L)
            except ValueError:
                result[(B1,B2)] = f"No D-finite recurrence found with {bound_guess} data"
    # case where only 1 pair is required
    if len(result) == 1:
        return result[(first_bases[0], second_bases[0])]
    return result

In [64]:
## METHODS RELATED WITH OEIS
def extract_recurrence(formula, neg_order, verbose=False):
    import re
    parts = [el.strip() for el in formula.split(".")[0].split(",")]
    ## extracting initial values
    #init_reg = r"a\((-?(?:\d+))\) ?= ?(-?(?:\d+))"
    #inits = {int(el[0]) : int(el[1]) for el in sum([re.findall(init_reg, el) for el in parts], [])}
    
    # substitution reg
    regs = [(r"a\(n\+(\d+)\)", r"E**(\1 + "+str(neg_order)+")"),
        (r"a\(n-(\d+)\)", r"E**(-\1 + "+str(neg_order)+")"),
        (r"a\(n\)", r"E**("+str(neg_order)+")"),
        (r"(\d+)n", r"\1*n")]
    def __apply_iterative(regs, string):
        for reg in regs:
            string = re.sub(*reg, string)
        return string
    for part in parts:
        part = part.replace(":", "")
        if part.find(" = ") > -1:
            try:
                lhs, rhs = part.split(" = ")
                lhs = __apply_iterative(regs, lhs)
                lhs = lhs.replace("n", f"(x+{neg_order})")
                rhs = __apply_iterative(regs, rhs)
                rhs = rhs.replace("n", f"(x+{neg_order})")
            
                return OE(lhs) - OE(rhs)#, inits
            except:
                if verbose: print(f"Can not convert this: {part} (({lhs} ||| {rhs}))")
                pass
    return None

def search_dfinite_order(min_order, min_results = 5, i = 0, verbose=False):
    import re
    shift = i+1
    results = []
    with alive_bar(int(min_results), title="Searching examples...", force_tty=True) as bar:
        while(len(results) < min_results):
            if verbose: print(f"Cathcing {i*100} first sequences")
            sequences = oeis("D-finite", max_results = 100, first_result = (i-1)*100)
            if len(sequences) == 0:
                raise KeyboardInterrupt("No more D-finite sequences")
            if verbose: print(f"Analyzing...")
            if verbose: print("Found orders ", end = "")
            for sequence in sequences:
                formulas = [el for el in sequence.formulas() if el.find("D-finite") >= 0]
                for formula in formulas:
                    try:
                        start_pos = formula.find("D-finite with recurrence: ") + len("D-finite with recurrence: ")
                        end_pos = max(formula.find(".", start_pos), len(formula))
                        formula = formula[start_pos:end_pos+1]

                        n = var('n')
                        arguments = [eval(el)-n for el in re.findall(r"a\(([^\)]*)\)", formula)]
                        arguments = [ZZ(el) for el in arguments if el in ZZ]

                        if(len(arguments) > 0):
                            order = max(arguments)-min(arguments)
                            neg_order = -min(0, min(arguments))
                            if(order >= min_order):
                                operator = extract_recurrence(formula, neg_order, verbose=verbose)
                                if operator != None:
                                    results.append((sequence, operator, solution(operator, sequence.first_terms(required_init(operator)+1))))
                                    bar()
                        else:
                            if verbose: print("no args", end=", ")
                    except:
                        pass
            i += 1
    return results, shift

In [65]:
dfinite, shift = search_dfinite_order(3,min_results=50)

Searching examples... |████████████████████████████████████████✗︎ (!) 57/50 [114%] in 5.4s (10.62/s)                     


In [28]:
explored = []
with alive_bar(len(dfinite), title="Exploring examples...", force_tty=True) as bar:
    for el in dfinite:
        explored.append(explore_example(el[1], el[0].first_terms(required_init(el[1])+1), OE, B, B))
        bar()

Exploring examples... |████████████████████████████████████████| 65/65 [100%] in 10:12.0 (0.11/s)                       


In [52]:
with_oeis = []
with alive_bar(len(explored), title="Checking if new is in OEIS...", force_tty=True) as bar:
    for el in explored:
        if (not isinstance(el, str)) and len(oeis([el[0](i) for i in range(10)])) > 0:
            with_oeis.append(el)
        bar()

Checking if new is in OEIS... |████████████████████████████████████████| 65/65 [100%] in 16.2s (4.02/s)                 


In [69]:
## Infinite search
found_dfinite = []
shift = 0
explored = []
with_oeis = []
analyzed = 0
while(True):
    try:
        dfinite, shift = search_dfinite_order(3,min_results=50,i=shift)
        found_dfinite.extend(dfinite)
        with alive_bar(len(dfinite), title="Exploring examples...", force_tty=True) as bar:
            for el in dfinite:
                explored.append(explore_example(el[1], el[0].first_terms(required_init(el[1])+1), OE, B, B))
                bar()
        with alive_bar(len(dfinite), title="Checking if new is in OEIS...", force_tty=True) as bar:
            for i in range(len(dfinite)):
                if(not isinstance(explored[-i-1], str)) and len(oeis([explored[-i-1][0](j) for j in range(10)])) > 0:
                    with_oeis.append(explored[-i-1])
                bar()
                
        print(f"Found {len(with_oeis)}/{len(explored)} cases with a sequence in OEIS")
    except KeyboardInterrupt:
        break

Searching examples... |████████████████████████████████████████✗︎ (!) 57/50 [114%] in 9.4s (6.08/s)                      
Exploring examples... |████████████████████████████████████████| 57/57 [100%] in 8:21.1 (0.11/s)                        
Checking if new is in OEIS... |████████████████████████████████████████| 57/57 [100%] in 5.7s (10.03/s)                 
Found 6/57 cases with a sequence in OEIS
Searching examples... |████████████████████████████████████████✗︎ (!) 65/50 [130%] in 9.9s (6.56/s)                      
Exploring examples... |████████████████████████████████████████| 65/65 [100%] in 9:20.5 (0.12/s)                        
Checking if new is in OEIS... |████████████████████████████████████████| 65/65 [100%] in 5.0s (12.97/s)                 
Found 11/122 cases with a sequence in OEIS
Searching examples... |████████████████████████████████████████✗︎ (!) 52/50 [104%] in 7.7s (6.74/s)                      
Exploring examples... |████████████████████████████████████████| 5

ZeroDivisionError: rational division by zero

In [90]:
print("Possible examples:")
interesting_cases = list(set([str(oeis([[el for el in with_oeis if el[1].order() > 1][j][0](i) for i in range(10)])).split(":")[1].strip() for j in range(10)]))
interesting_cases

Possible examples:


['A007317', 'A000172', 'A258431', 'A032443']

### Trying some examples with no-Liouvillian solutions

We know that the convolution of $n!$ and $1/n!$ is not Liouvillian and satisfies the following recurrence:
$$(n+3)y_{n+3} - (n^2 + 6n + 10)y_{n+2} + (2n + 5)y_{n+1} - y_n  = 0.$$

Let us see what happen if we apply the binomial basis to this recurrence:

In [67]:
non_liouv = lambda n : sum(QQ(factorial(k)/factorial(n-k)) for k in range(n+1));
show([non_liouv(i) for i in range(10)])
# Recurrence for the sequence
L = (x+3)*E^3 - (x^2 + 6*x + 10)*E^2 + (2*x + 5)*E - 1
[apply_operator_to_seq(L, non_liouv)(i) for i in range(10)]

[1,
 2,
 7/2,
 26/3,
 749/24,
 8843/60,
 614227/720,
 3669289/630,
 1844279257/40320,
 10519507811/25920]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [68]:
# Getting recurrence by adding a binomial factor
L2 = ass_recurrence(L, B, OE); show(L2)
# Getting the solution of the sequence
non_liouv_bin = solution(L2,get_converted_init(non_liouv, B, 6)) 
[non_liouv_bin(i) for i in range(10)]

(x + 5)*E^5 + (-x^2 - 6*x - 9)*E^4 + (-4*x^2 - 25*x - 40)*E^3 + (-6*x^2 - 31*x - 41)*E^2 + (-4*x^2 - 16*x - 16)*E - x^2 - 3*x - 2

[1,
 1,
 1/2,
 19/6,
 253/24,
 6241/120,
 219661/720,
 10574971/5040,
 667791769/40320,
 53505430273/362880]

In [69]:
# Getting recurrence by adding a second binomial factor
L3 = ass_recurrence(L, B, OE); show(L3)
# Getting the solution of the sequence
non_liouv_bin_bin = solution(L2,get_converted_init(non_liouv_bin, B, 6)) 
[non_liouv_bin_bin(i) for i in range(10)]

(x + 5)*E^5 + (-x^2 - 6*x - 9)*E^4 + (-4*x^2 - 25*x - 40)*E^3 + (-6*x^2 - 31*x - 41)*E^2 + (-4*x^2 - 16*x - 16)*E - x^2 - 3*x - 2

[1,
 0,
 -1/2,
 11/3,
 -17/8,
 2617/120,
 56437/720,
 3045667/5040,
 189260353/40320,
 15193219561/362880]

In [74]:
guess([non_liouv_bin_bin(i) for i in range(50)], OE)

(-x - 5)*E^5 + (x^2 + 6*x + 9)*E^4 + (4*x^2 + 25*x + 40)*E^3 + (6*x^2 + 31*x + 41)*E^2 + (4*x^2 + 16*x + 16)*E + x^2 + 3*x + 2

### Trying to use guess to obtain an example

In [203]:
a = -1; b = -2; c = -3
seq = lambda n : solution(E^2 - (x-a)*(x-b)*E - (x-a)*(x-b)*(x-c), [1,1])(n)
bin_seq = lambda n : sum(binomial(n,k)*seq(k) for k in range(n+1))
bin_bin_seq = lambda n : sum(binomial(n,k)*bin_seq(k) for k in range(n+1))

In [204]:
[seq(i) for i in range(10)]

[1, 1, 8, 72, 1344, 35520, 1347840, 68544000, 4517775360, 374631505920]

In [205]:
[bin_bin_seq(i) for i in range(20)]

[1,
 3,
 16,
 140,
 2160,
 52592,
 1868416,
 90820032,
 5783000320,
 466805000960,
 46565899505664,
 5624284106058752,
 808786822879473664,
 136539590048636497920,
 26739228138809412812800,
 6012248327840046068645888,
 1538283273936198105335660544,
 444360163131793080333495369728,
 143917487573718575394901511372800,
 51937374756541740876020471330242560]

### The example [A001472](https://oeis.org/A001472)

This sequence starts with the terms $1, 1, 2, 4, 16, 56, 256, 1072, 6224, 33616, 218656,\ldots$ and counts the number of degree-$n$ permutations of order dividing 4. It satisfies a recurrence: 
$$a(0)=1, a(1)=1, a(2)=2, a(3)=4,$$
$$a(n) = a(n-1) + (n-1)a(n-2) + (n^3-6n^2+11n-6)a(n-4)$$

In [34]:
L001472 = E^4 - E^3 - (x+3)*E^2 - ((x+4)^3 - 6*(x+4)^2 + 11*(x+4) - 6)
A001472 = solution(L001472, [1,1,2,4])
[A001472(i) for i in range(11)]

[1, 1, 2, 4, 16, 56, 256, 1072, 6224, 33616, 218656]

In [35]:
seq, op = explore_example(L001472, [1,1,2,4], OE, B, B)

In [36]:
[seq(i) for i in range(20)]

[1,
 3,
 10,
 36,
 144,
 648,
 3264,
 18000,
 107088,
 682992,
 4657824,
 33763392,
 258543360,
 2082340224,
 17600910336,
 155694742272,
 1436921254656,
 13798542792960,
 137635003878912,
 1423560240073728]

This sequence has a formula given as follows:

$$a(n) = n!\sum_{k=1}^n \frac{1}{k!}\left(\sum_{j=\lfloor(4*k-n)/3\rfloor}^k\binom{k}{j} \binom{j}{n-4k+3j}(1/2)^{n-4k+3j}(1/4)^{k-j}\right)$$

In [219]:
bin_A001472 = lambda n : sum(binomial(n,k)*A001472(k) for k in range(n+1))
Lbin_001472 = guess([bin_A001472(i) for i in range(50)], OE)
bin_bin_A001472 = lambda n : sum(binomial(n,k)*bin_A001472(k) for k in range(n+1))
Lbin_bin_001472 = guess([bin_bin_A001472(i) for i in range(50)], OE)
show(L001472)
show(Lbin_001472)
print(latex(-Lbin_bin_001472).replace("x", "n"))
[bin_bin_A001472(i) for i in range(10)]

E^4 - E^3 + (-x - 3)*E^2 - x^3 - 6*x^2 - 11*x - 6

-E^4 + 2*E^3 + (x + 3)*E^2 + x^3 + 6*x^2 + 11*x + 6

E^{4} - 3 E^{3} + \left(-n - 3\right) E^{2} - n^{3} - 6 n^{2} - 11 n - 6


[1, 3, 10, 36, 144, 648, 3264, 18000, 107088, 682992]

#### Writing the example

Consider the sequence $(1,3,10,36,144,648,3264,18000,107088,682992,\ldots)$ defined as solution to the recurrence equation:
$$y_{n+4} - 3 y_{n+3} + \left(-n - 3\right) y_{n+2} - (n^{3} + 6 n^{2} + 11 n + 6)y_n = 0$$

This sequence does not show up in OEIS, but if we write it in the following form:
$$y_n = \sum_{k=0}^n c_k \binom{n}{k},$$
then the sequence $(c_n)_n$ also satisfies a recurrence equation given by:

In [229]:
equ_y = -Lbin_bin_001472
seq_y = solution(equ_y, [1,3,10,36, 144])
equ_c = ass_recurrence(equ_y, B, OE); show(equ_c)
seq_c = solution(equ_c,get_converted_init(seq_y, B, 7)) 
[seq_c(i) for i in range(10)]

E^7 + E^6 + (-x - 9)*E^5 + (-3*x - 20)*E^4 + (-x^3 - 15*x^2 - 77*x - 134)*E^3 + (-3*x^3 - 36*x^2 - 142*x - 183)*E^2 + (-3*x^3 - 27*x^2 - 78*x - 72)*E - x^3 - 6*x^2 - 11*x - 6

[1, 2, 5, 14, 49, 202, 949, 4790, 26513, 159218]

This sequence does not appear in OEIS still. The order has gotten bigger and we still have no information. We can try to apply the transformation once again:
$$c_n = \sum_{k=0}^nd_k\binom{n}{k}.$$ 
This sequence $(d_n)_n$ is also D-finite. Let see the recurrence and solution:

In [232]:
equ_d = ass_recurrence(equ_c, B, OE); show(equ_d)
seq_d = solution(equ_d,get_converted_init(seq_c, B, 10)) 
[seq_d(i) for i in range(10)]

E^10 + 8*E^9 + (-x + 15)*E^8 + (-9*x - 42)*E^7 + (-x^3 - 24*x^2 - 224*x - 732)*E^6 + (-9*x^3 - 189*x^2 - 1377*x - 3432)*E^5 + (-33*x^3 - 594*x^2 - 3597*x - 7288)*E^4 + (-63*x^3 - 945*x^2 - 4698*x - 7712)*E^3 + (-66*x^3 - 792*x^2 - 3110*x - 3984)*E^2 + (-36*x^3 - 324*x^2 - 936*x - 864)*E - 8*x^3 - 48*x^2 - 88*x - 48

[1, 1, 2, 4, 16, 56, 256, 1072, 6224, 33616]

This sequence does appear in OEIS as the sequence [A001472](https://oeis.org/A001472). Sequence A001472 is D-finite with recurrence
$$a(n) = a(n-1) + (n-1)a(n-2) + (n^3-6n^2+11n-6)a(n-4).$$
Using closure properties, we can check if these two sequences are the same:

In [240]:
L001472 = E^4 - E^3 - (x+3)*E^2 - ((x+4)^3 - 6*(x+4)^2 + 11*(x+4) - 6)
A001472 = solution(L001472, [1,1,2,4])
diff_equ = L001472.lclm(equ_d)
diff_seq = solution(diff_equ, [A001472(i) - seq_d(i) for i in range(diff_equ.order()+5)])
show(diff_equ)
[diff_seq(i) for i in range(diff_equ.order()+1)]

E^10 + 8*E^9 + (-x + 15)*E^8 + (-9*x - 42)*E^7 + (-x^3 - 24*x^2 - 224*x - 732)*E^6 + (-9*x^3 - 189*x^2 - 1377*x - 3432)*E^5 + (-33*x^3 - 594*x^2 - 3597*x - 7288)*E^4 + (-63*x^3 - 945*x^2 - 4698*x - 7712)*E^3 + (-66*x^3 - 792*x^2 - 3110*x - 3984)*E^2 + (-36*x^3 - 324*x^2 - 936*x - 864)*E - 8*x^3 - 48*x^2 - 88*x - 48

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Sequnce A001472 has a closed form formula. Hence, we can write the solutions to our original equation
$$y_{n+4} - 3 y_{n+3} + \left(-n - 3\right) y_{n+2} - (n^{3} + 6 n^{2} + 11 n + 6)y_n = 0$$
with the following formula:
$$y_n = \sum_{k=0}^n 
    \sum_{l=0}^k 
        \left(l!\sum_{p=1}^l 
            \frac{1}{p!}\left(
                \sum_{j=\lfloor(4p-l)/3\rfloor}^p\binom{p}{j} \binom{j}{l-4p+3j}(1/2)^{l-4p+3j}(1/4)^{p-j}
            \right)
        \right)
\binom{k}{l}\binom{n}{k}$$